**STEPS TO FOLLOW**

- Pick the Active Eligible table from config Table where active_flag=1, load_flag=1 and bq_to_gcs_status is 'COMPLETED'
- Reads the parqut file from GCS (GCS path is like gcs_path/dt=/*.parquet) means take all the files present inside gcs path for any particular timestamp 
- Uses the latest dt(datetime file) folder
- Writes the delta bronze path
- Create the Table from bronze path to bronze dataset 
- Update the config table

In [0]:
dbutils.widgets.text("mysql_host","34.56.24.205")
mysql_host = dbutils.widgets.get("mysql_host")

dbutils.widgets.text("mysql_port","3306")
mysql_port = dbutils.widgets.get("mysql_port")

dbutils.widgets.text("mysql_root","root")
mysql_user = dbutils.widgets.get("mysql_root")

dbutils.widgets.text("mysql_db","GCPMigrationMeta")
mysql_db = dbutils.widgets.get("mysql_db")

dbutils.widgets.text("mysql_password","Anupamnaina#18")
mysql_password = dbutils.widgets.get("mysql_password")

dbutils.widgets.text("bq_sa_key",'/Volumes/workspace/default/csv/datamigrationproject-483310-739969975183.json')
GOOGLE_APPLICATION_CREDENTIALS = dbutils.widgets.get("bq_sa_key")

dbutils.widgets.text("catalog_name", "bronze")
catalog_name = dbutils.widgets.get("catalog_name")

dbutils.widgets.text("schema_name", "raw")
schema_name = dbutils.widgets.get("schema_name")

In [0]:
%pip install mysql-connector-python google-cloud-bigquery google-cloud-storage

In [0]:
import json, os, datetime as dt
import mysql.connector as mc
from google.cloud import bigquery
from contextlib import contextmanager 
# contextlib python library help us to setup and clean up the resources automatically

MYSQL = {
    'host':mysql_host.strip(),
    'port':int(mysql_port),
    'user':mysql_user.strip(),
    'pwd':mysql_password,
    'db' :mysql_db
}
BQ_KEY=GOOGLE_APPLICATION_CREDENTIALS.replace('dbfs:/','/dbfs/') 
os.environ("GOOGLE_APPLICATION_CREDENTIALS") = BQ_KEY

In [0]:
@contextmanager
def mysql_conn():
    conn = mc.connect(
        host = MYSQL['host'], 
        port = MYSQL['port'],
        user = MYSQL['user'],
        password = MYSQL['pwd'],
        database = MYSQL['db']
    )
    try:
        yield conn
    except Exception as e:
        print(f"Failed to connect to MYSQL \nHere is the error -: \n{e}")
    # yield returns a value temporarily and pauses the function, then resumes later from the same place.
    finally:
        conn.close()

In [0]:
def fetch_eligible_tables():
    with mysql_conn() as conn:
        cur = conn.cursor()
        cur.execute("Select * from GCPMigrationMeta.config_table where active_flag=1 and load_flag=1")
        rows = cur.fetchall()
        cur.close()
    return rows

fetch_eligible_tables()

In [0]:
def set_bronze_status(table_name, status, err = None):
     with mysql_conn() as conn:
        cur = conn.cursor()
        if status == 'IN_PROGRESS':
            # Mark as in progress and clear any previous error 
            cur.execute("""UPDATE GCPMigrationMeta.config_table set gcs_to_bronze_status="IN_PROGRESS", last_run_ts=NOW(), error_message=NULL where table=%s""", (table_name))

        elif status == 'COMPLETED':
            # Mark as in progress and clear any previous error 
            cur.execute("""UPDATE GCPMigrationMeta.config_table set gcs_to_bronze_status="COMPLETED", last_run_ts=NOW(), error_message=NULL where table=%s""", (table_name))

        else:
            # Mark as in progress and clear any previous error 
            cur.execute("""UPDATE GCPMigrationMeta.config_table set gcs_to_bronze_status="FAILED", last_run_ts=NOW(), error_message = '{str(err)[:2000] if err else "FAILED"},                     WHERE table_name = '{table_name}')    

        

In [0]:
def reset_load_flag(table_name):
    with mysql_conn() as conn:
        cur = conn.cursor()
        cur.execute("""UPDATE CPMigrationMeta.config_table set load_flag=0 where table_name=%s""", (table_name))
        conn.commit()
        cur.close()

In [0]:
# This will return us the bucket name
def gcs_client():
    # storage is a gcs inbuilt fun that will give us the storage details
    return storage.Client()